In [15]:
import pdfplumber
import re
import json
import os
import PyPDF2


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

## METODI

In [16]:
file_path = "../Hackapizza Dataset/Codice Galattico/Codice Galattico.pdf"

load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [17]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [28]:
def estrai_sostanze(llm: AzureChatOpenAI, pdf_path: str) -> Dict[str, str]:
    """
    Estrae le sostanze e le loro proprietà principali da un PDF, analizzando la sezione 2.2.

    Args:
        llm (AzureChatOpenAI): Istanza del modello linguistico.
        pdf_path (str): Percorso al file PDF.

    Returns:
        Dict[str, str]: Dizionario con il nome della sostanza come chiave e le proprietà principali come valore.
    """
    
    testo_sezione = ""
    
    # Legge il PDF ed estrae testo e tabelle
    with pdfplumber.open(pdf_path) as doc:
        for page in doc.pages:
            text = page.extract_text()
            if text and "2.2" in text:  # Trova la sezione 2.2
                testo_sezione += text  # Aggiunge tutto il testo rilevante
            
            # Estrai tabelle
            tables = page.extract_tables()
            for table in tables:
                for row in table:
                    # Sostituisci eventuali valori None con una stringa vuota
                    row_cleaned = [str(cell) if cell is not None else "" for cell in row]
                    testo_sezione += " | ".join(row_cleaned) + "\n"

    if not testo_sezione:
        print("Sezione 2.2 non trovata nel PDF.")
        return {}

    # Prompt migliorato per il modello AI
    prompt = f"""
    Analizza il seguente testo e identifica tutte le sostanze con le loro proprietà principali.

    La tabella ha più colonne:
    - La prima colonna contiene il nome della sostanza
    - Le altre colonne contengono le proprietà principali, con i relativi valori

    Restituisci un oggetto JSON con:
    - La chiave come il nome della sostanza
    - Il valore come una stringa con tutte le proprietà principali e i loro valori (es. "CRP: 0.89, IEI: 0.3")

    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.

    Testo:
    {testo_sezione}
    """

    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        sostanze = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle sostanze:", content)
        sostanze = {}

    return sostanze


In [31]:
def aggiungi_sostanza(tx, nome: str, proprieta: str):
    """
    Aggiunge una sostanza e le sue proprietà principali al database Neo4j.
    
    Args:
        tx: Transazione Neo4j
        nome (str): Nome della sostanza
        proprieta (str): Proprietà principale (può essere una stringa con più proprietà separate da virgola)
    """
    query = """
    MERGE (s:Sostanza {nome: $nome})
    SET s.proprieta = $proprieta
    """
    tx.run(query, nome=nome, proprieta=proprieta)

## MAIN

### SOSTANZE

In [29]:
sostanze = estrai_sostanze(llm, file_path)

In [30]:
print(sostanze)

{'Erba Pipa': 'CRP: 0.89, IEI: 0.3', 'Cristalli di Memoria': 'CRP: 0.92, CDT: 0.2', 'Petali di Eco': 'CRP: 0.87, IPM: 0.6', 'Carne di Drago': 'IPM: 0.85, IBX: 0.75', 'Uova di Fenice': 'IPM: 0.98, CDT: 0.8', 'Lacrime di Unicorno': 'IPM: 0.95, θ: 0.9', 'Foglie di Mandragora': 'IBX: 0.82, μ: 0.3', 'MuNa Lunare': 'IBX: 0.78, μ: 0.6, CRP: 0.72', 'Nettare di Sirena': 'IBX: 0.85, μ: 0.2, θ: 0.7', 'Spore Quantiche': 'δQ: 0.45, ID: 0.8', 'Essenza di Vuoto': 'δQ: 0.16, ID: 0.95', 'Funghi dell’Etere': 'δQ: 0.38, ID: 0.75', 'Sale Temporale': 'CDT: 0.65, IEI: 0.4', 'Radici di Gravità': 'CDT: 0.55, ID: 0.4', 'Polvere di Stelle': 'CDT: 0.75, IPM: 0.88'}


In [ ]:
with driver.session() as session:
        for nome, proprieta in sostanze.items():
            session.execute_write(aggiungi_sostanza, nome, proprieta)

C:\Users\lparigi\AppData\Local\Temp\ipykernel_29264\3805961789.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(aggiungi_sostanza, nome, proprieta)


### CREA RELAZIONI TECNICA RICHIEDE LICENZA 

In [33]:
def estrai_testo_capitolo_4(pdf_path: str) -> str:
    """
    Estrae il testo della sezione 4 del PDF che contiene le descrizioni delle tecniche e licenze.

    Args:
        pdf_path (str): Percorso al file PDF.

    Returns:
        str: Il testo del capitolo 4.
    """
    testo_capitolo_4 = ""

    with pdfplumber.open(pdf_path) as doc:
        for page in doc.pages:
            testo = page.extract_text()
            if testo and "4." in testo:  # Cerca il capitolo 4
                testo_capitolo_4 += testo

    if not testo_capitolo_4:
        print("Sezione 4 non trovata nel PDF.")
    return testo_capitolo_4

testo = estrai_testo_capitolo_4(file_path)
print(testo)

789/12773 67° Giorno del Ciclo Cosmico 789
2. Ingredienti di Origine Mitica: sostanze derivate da creature leggendarie, la cui
esistenza trascende le normali categorie biologiche e richiede considerazioni sia
pratiche che etiche nella loro gestione.
3. Sostanze Xenobiologiche: materiali di origine biologica non terrestre che richiedono
particolare attenzione nella manipolazione e conservazione, con potenziali
interazioni con i sistemi biologici terrestri.
4. Sostanze Quantiche e Dimensionali: ingredienti che manifestano proprietà
quantistiche o multi-dimensionali, necessitando di specifiche precauzioni per
prevenire anomalie spazio-temporali.
5. Sostanze Spazio-Temporali: elementi che interagiscono con il tessuto spazio-
temporale, richiedendo particolare attenzione per evitare distorsioni locali della
realtà.
2.2 Tabella delle sostanze regolamentate
La presente tabella fornisce una panoramica delle principali categorie di sostanze
regolamentate.
Sostanza Proprietà principali
Sostanze 

In [ ]:
def estrai_tecniche_licenze(testo: str) -> list:
    """
    Estrae le tecniche e le licenze dal testo per la sezione del PDF riguardante la marinatura.

    Args:
        testo (str): Il testo contenente le tecniche e le licenze.

    Returns:
        list: Una lista di tuple contenenti il nome della tecnica e la licenza.
    """
    # Espressione regolare per estrarre la tecnica, la licenza e il livello
    pattern = r"\s*([A-Za-z\s]+(?:[a-zA-Z\s]+))\s*richiede\s*una\s*licenza\s*([A-Za-z]+)\s*\([a-zA-Z\+\-]+\)\s*di\s*livello\s*(I{1,3}|IV|[1-9]+)"

    tecniche_licenze = re.findall(pattern, testo)

    # Processa i risultati per formattare correttamente la tecnica e la licenza
    tecniche_licenze_formattate = []
    for tecnica, licenza, livello in tecniche_licenze:
        # Rimuovi l'articolo "La" dalla tecnica
        tecnica = tecnica.strip().replace("La ", "")
        
        # Capitalizza correttamente la tecnica (ogni parola inizia con una lettera maiuscola)
        tecnica = ' '.join([parola.capitalize() for parola in tecnica.split()])

        # Crea la licenza nel formato "Licenza livello X"
        licenza_formattata = f"{licenza.capitalize()} livello {livello}"

        tecniche_licenze_formattate.append((tecnica, licenza_formattata))

    return tecniche_licenze_formattate



tecniche_licenze = estrai_tecniche_licenze(testo)

In [45]:
print(tecniche_licenze)

[('Marinatura A Infusione Gravitazionale', 'Gravitazionale livello II'), ('Marinatura Temporale Sincronizzata', 'Temporale livello I'), ('Antimateria Diluite', 'Antimateria livello I'), ('Umicatura A Stratificazione Quantica', 'Quantistica livello 3'), ('Umicatura Psionica Sensoriale', 'Psionica livello II'), ('Umicatura Polarizzata A Freddo Iperbarico', 'Magnetica livello I'), ('Fermentazione Quantica A Strati Multiversali', 'Quantistica livello 5'), ('Fermentazione Psionica Energetica', 'Psionica livello I'), ('Fermentazione Quantico Biometrica', 'Quantistica livello 3'), ('Decostruzione Atomica A Strati Energetici', 'Antimaterica livello I'), ('Decostruzione Magnetica Risonante', 'Magnetica livello I'), ('Decostruzione Ancestrale', 'Temporale livello III'), ('Decostruzione Interdimensionale Lovercraftiana', 'Quantistica livello 7'), ('Psionica Variabile', 'Psionica livello I'), ('Sferificazione Filamentare A Molecole Vibrazionali', 'Magnetica livello I'), ('Sferificazione Cromatica 

In [46]:
def aggiungi_relazione_licenza(tx, tecnica: str, licenza: str):
    """
    Aggiunge una relazione RICHIEDE_LICENZA tra una tecnica e la licenza nel database Neo4j.

    Args:
        tx: Transazione Neo4j
        tecnica (str): Nome della tecnica
        licenza (str): Nome della licenza
    """
    query = """
    MATCH (t:Tecnica {nome: $tecnica}), (l:Licenza {nome: $licenza})
    MERGE (t)-[:RICHIEDE_LICENZA]->(l)
    """
    tx.run(query, tecnica=tecnica, licenza=licenza)

In [47]:
with driver.session() as session:
    for tecnica, licenza in tecniche_licenze:
        session.execute_write(aggiungi_relazione_licenza, tecnica, licenza)

### CAPITOLI 3, 4, 5

In [ ]:
capitolo_3 = pdf_chapters["Capitolo 3"]
capitolo_4 = pdf_chapters["Capitolo 4"]
capitolo_5 = pdf_chapters["Capitolo 5"]

macrotecniche, tecniche = estrai_tecniche_e_macro(llm, capitolo_5)

# Per visualizzare i risultati:
print("\nMacrotecniche:")
for macro, descrizione in macrotecniche.items():
    print(f"\n{macro}:")
    print(descrizione)

print("\nTecniche specifiche:")
for tecnica in tecniche:
    print(f"- {tecnica}")

In [ ]:
dettagli_per_grafo = estrai_dettagli_tecniche_per_grafo(llm, capitolo_5, tecniche)

# Inserimento dati nel database
with driver.session() as session:
    for macro, descrizione in macrotecniche.items():
        session.execute_write(aggiungi_macrotecniche, macro, descrizione)

print("Macrotecniche aggiunte con successo!")

# Inserire le tecniche nel database
with driver.session() as session:
    for nome, dettagli in dettagli_per_grafo.items():
        session.execute_write(
            aggiungi_tecniche, nome, dettagli["come_funziona"], dettagli["vantaggi"], dettagli["svantaggi"]
        )

print("Tecniche aggiunte con successo!")